In [1]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit import QuantumRegister, ClassicalRegister


provider = QiskitRuntimeService()          
backend = provider.backend('ibmq_qasm_simulator')


pi = np.pi

q = QuantumRegister(1,'q')
c = ClassicalRegister(1,'c')

circuit = QuantumCircuit(q,c)

circuit.ry(pi,q[0])
circuit.measure(q,c)

print(circuit)

new_circuit = transpile(circuit, 
                        backend = backend)

job = backend.run(new_circuit,
                  shots = 1024)

counts = job.result().get_counts()

print(counts)

     ┌───────┐┌─┐
  q: ┤ Ry(π) ├┤M├
     └───────┘└╥┘
c: 1/══════════╩═
               0 
{'1': 1024}


### VQE

https://qiskit-community.github.io/qiskit-optimization/tutorials/06_examples_max_cut_and_tsp.html

In [2]:
import time
import numpy as np
from qiskit_aer import Aer
from scipy.optimize import minimize
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile

##### Construction of the Hamiltonian

In [3]:
def qubo_to_ising(input_Q):

    # Define the 2x2 matrices we need

    # (1 + pauli_z)/2
    sigma_z = np.array([[1, 0],
                        [0, 0]])

    # (1 - sigma_z)
    minus_z = np.array([[0, 0],
                        [0, 1]])

    # Identity
    id_matrix = np.array([[1, 0],
                          [0, 1]])

    n = len(input_Q)
    print("input:")
    print(input_Q)
    print("")
    
    # initialize H
    H = 0

    # compute the contribution of the i,j term to the Hamiltonian
    # i = left-side term = x_i (corresponds to sigma_z)
    for i in range(n):
        # j = right-side term = (1 - x_j) (corresponds to minus_z)
        for j in range(n):            
            # first term
            matrix_ij = 0
            if i == 0:
                matrix_ij = sigma_z
            elif j == 0:
                matrix_ij = minus_z
            else:
                matrix_ij = id_matrix
            
            # tensor product n times
            for k in range(1,n):
                if i == k:
                    new_term = sigma_z
                elif j == k:
                    new_term = minus_z
                else:
                    new_term = id_matrix                
                matrix_ij = np.kron(matrix_ij, new_term)

            # multiply by the i,j term of input_Q 
            matrix_ij = matrix_ij * input_Q[i,j]
            
            # sum
            H = H + matrix_ij

    print(H)
    return(-H)

In [4]:
input_Q = np.array([[-1/(2*1**2),           0],
                    [          0, -1/(2*2**2)]])

qubo_to_ising(input_Q)

input:
[[-0.5    0.   ]
 [ 0.    -0.125]]

[[-0.625  0.     0.     0.   ]
 [ 0.    -0.5    0.     0.   ]
 [ 0.     0.    -0.125  0.   ]
 [ 0.     0.     0.     0.   ]]


array([[ 0.625, -0.   , -0.   , -0.   ],
       [-0.   ,  0.5  , -0.   , -0.   ],
       [-0.   , -0.   ,  0.125, -0.   ],
       [-0.   , -0.   , -0.   , -0.   ]])

##### Cost function

In [5]:
def cost_function_C(results, weights):
    
    # the eigenstates obtained by the evaluation of the circuit
    eigenstates = list(results.keys())
    
    # how many times each eigenstate has been sampled
    abundancies = list(results.values())
    
    # number of shots 
    shots = sum(results.values())
    
    # initialize the cost function
    cost = 0
    
    for k in range(len(eigenstates)):
        # ndarray of the digits extracted from the eigenstate string 
        x = np.array([int(num) for num in eigenstates[k]])
        # Cost function due to the k-th eigenstate
        cost = cost + x.dot(weights.dot(1-x)) * abundancies[k]
    
    return -cost / shots

##### Definition of the VQE_circuit

https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.library.RYGate

https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.library.CZGate

In [6]:
def VQE_circuit(theta, n, depth): 
    """
    Creates a variational-form RY ansatz.
    
    theta: (depth+1 x n) matrix of rotation angles.
    n    : number of qbits.
    depth: number of layers.
    """
        
    if len(theta.ravel()) != ((depth+1) * n):        
        raise ValueError("Theta cannot be reshaped as a (depth+1 x n) matrix")

    theta.shape = (depth + 1, n)

    # Define the Quantum and Classical Registers
    q = QuantumRegister(n)
    c = ClassicalRegister(n)

    # Build the circuit for the ansatz
    circuit = QuantumCircuit(q, c)

    # Put all the qbits in the |+> state
    for i in range(n):
        circuit.ry(theta[0,i],q[i])
    circuit.barrier()
    
    # Now introduce the z-gates and RY-gates 'depth' times
    for j in range(depth):
        # Apply controlled-z gates
        for i in range(n-1):
            circuit.cz(q[i], q[i+1])

        # Introduce RY-gates
        for i in range(n):
            circuit.ry(theta[j+1,i],q[i])
        circuit.barrier()
    
    # Close the circuit with qbits measurements
    circuit.measure(q, c)
    
    return circuit

In [7]:
theta = 0

theta_matrix = np.array([[np.cos(theta/2), -np.sin(theta/2)],
                         [np.sin(theta/2),  np.cos(theta/2)]])

print(theta_matrix)

n = 2
depth = 1
vqe = VQE_circuit(theta_matrix, n, depth)

vqe.draw()

[[ 1. -0.]
 [ 0.  1.]]


┌───────┐ ░    ┌───────┐ ░ ┌─┐   
q3_0: ┤ Ry(1) ├─░──■─┤ Ry(0) ├─░─┤M├───
      ├───────┤ ░  │ ├───────┤ ░ └╥┘┌─┐
q3_1: ┤ Ry(0) ├─░──■─┤ Ry(1) ├─░──╫─┤M├
      └───────┘ ░    └───────┘ ░  ║ └╥┘
c0: 2/════════════════════════════╩══╩═
                                  0  1

In [9]:
provider = QiskitRuntimeService()          
backend = provider.backend('ibmq_qasm_simulator')

new_circuit = transpile(vqe, 
                        backend = backend)

job = backend.run(new_circuit,
                  shots = 1024)

counts = job.result().get_counts()

print(counts)

{'11': 56, '01': 175, '00': 603, '10': 190}


In [10]:
weights = np.array([[0, 1],
                    [1, 0]])

cost_function_C({'01': 189, '11': 48, '10': 182, '00': 605}, weights)

-0.3623046875

##### Minimization

In [9]:
def cost_function_cobyla(params, 
                         weights,   # = W, 
                         n_qbits      = 2, 
                         depth        = 1,
                         shots        = 1024,
                         cost         = 'cost',
                         algorithm    = "VQE", 
                         alpha        = 0.5,
                         backend_name = 'qasm_simulator',
                         verbosity    = False):
    """
    Creates a circuit, executes it and computes the cost function.
    
    params: ndarray with the values of the parameters to be optimized,
    weights: the original QUBO matrix of the problem,
    n_qbits: number of qbits of the circuit,
    depth: number of layers of the ciruit,
    shots: number of evaluations of the circuit state,
    cost: the cost function to be used. It can be: 
     - 'cost': mean value of all measured eigenvalues
     - 'cvar': conditional value at risk = mean of the
               alpha*shots lowest eigenvalues,
    alpha: 'cvar' alpha parameter
    verbosity: activate/desactivate some control printouts.
    
    The function calls 'VQE_circuit' to create the circuit, then
    evaluates it and compute the cost function.
    """
    
    if (verbosity == True):
        print("Arguments:")  
        print("params    = \n", params)
        print("weights   = \n", weights)
        print("qbits     = ", n_qbits)
        print("depth     = ", depth)
        print("shots     = ", shots)
        print("cost      = ", cost)
        print("algorithm = ", algorithm)
        print("alpha     = ", alpha)
        print("backend   = ", backend_name)
    
    circuit = VQE_circuit(params, n_qbits, depth)

    print(circuit.draw())
    
    if backend_name == 'qasm_simulator':
        backend = Aer.get_backend('qasm_simulator')
    else:
        provider = QiskitRuntimeService()
        backend = provider.backend(backend_name)
    
    # Execute the circuit on a simulator
    new_circuit = transpile(circuit, 
                            backend = backend)

    job = backend.run(new_circuit,
                      shots = shots)
    
    results = job.result()
 
    if cost == 'cost':
        output = cost_function_C(results.get_counts(), weights)
    #elif cost == 'cvar':
        # output = cv_a_r(results.get_counts(), weights, alpha)
    else:
        raise ValueError("Please select a valid cost function")
    
    if (verbosity == True):
        print("cost = ", output)
        print(results.get_counts(circuit))

    return output

weigths?

In [11]:
params = theta_matrix

weights = np.array([[0, 1],
                    [1, 0]])

cost_function_cobyla(params, weights, verbosity = True)

Arguments:
params    = 
 [[ 1. -0.]
 [ 0.  1.]]
weights   = 
 [[0 1]
 [1 0]]
qbits     =  2
depth     =  1
shots     =  1024
cost      =  cost
algorithm =  VQE
alpha     =  0.5
backend   =  qasm_simulator
      ┌───────┐ ░    ┌───────┐ ░ ┌─┐   
q9_0: ┤ Ry(1) ├─░──■─┤ Ry(0) ├─░─┤M├───
      ├───────┤ ░  │ ├───────┤ ░ └╥┘┌─┐
q9_1: ┤ Ry(0) ├─░──■─┤ Ry(1) ├─░──╫─┤M├
      └───────┘ ░    └───────┘ ░  ║ └╥┘
c2: 2/════════════════════════════╩══╩═
                                  0  1 
cost =  -0.337890625
{'11': 58, '01': 161, '10': 185, '00': 620}


-0.337890625

##### Time studies

In [8]:
def time_vs_shots(weights,
                  final_eval_shots,
                  cost         = 'cost',
                  shots        = 1024,
                  n_qbits      = 2,
                  depth        = 1,
                  backend_name = 'qasm_simulator',
                  alpha        = 0.5,
                  algorithm    = "VQE",
                  method       = "COBYLA",
                  theta        = 1,
                  verbosity    = False):
    """
    Returns the time taken to solve a VQE problem
    as a function of the shots.    
    
    Input parameters:
    shots: number of evaluations of the circuit state,
    weights: the original QUBO matrix of the problem,
    n_qbits: number of qbits of the circuit,
    depth: number of layers of the ciruit,
    backend_name: the name of the device where the optimization will be performed,
    final_eval_shots: number of shots for the evaluation of the optimized circuit,
    cost: the cost function to be used. It can be: 
     - 'cost': mean value of all measured eigenvalues
     - 'cvar': conditional value at risk = mean of the
               alpha*shots lowest eigenvalues,
    alpha: 'cvar' alpha parameter
    algorithm: the optimization algorithm to be used (VQE or QAOA),
    method: the classical optimizar (COBYLA or SLSQP),
    theta: the ansatz initial parameters. If set to 1, the 
        standard ry ansatz parameters are used,
    verbosity: activate/desactivate some control printouts.
    
    Output:
    elapsed_time: time taken for the optimization (in seconds)
    counts: dictionaty the results of the optimization
    shots: the 'shots' input parameter (it may be useful for analysis)
    n_func_evaluations: number of evaluations of the cost function
    final_eval_shots: shots for the optimal circuit evaluation
    optimal_angles: the theta parameters given by the optimization,
    final_cost: the cost function of the optimal circuit.
    
    """
    # Do this only if no initial parameters have been given
    if isinstance(theta, (int)):
        # Create the rotation angles for the ansatz
        theta_0       = np.repeat(np.pi/2, n_qbits)
        theta_0.shape = (1, n_qbits)
        theta_1       = np.zeros((depth, n_qbits))
        theta         = np.concatenate((theta_0, theta_1), axis = 0) 
    
    # Time starts with the optimization
    start_time = time.time()

    # print("method: {0}".format(method))

    # Classical optimizer tuning - COBYLA
    res = minimize(fun     = cost_function_cobyla, 
                   x0      = theta.ravel(),       # the 'params' argument of 'cost_function_cobyla'
                   method  = method, #'COBYLA',            # we want to use the COBYLA optimization algorithm
                   options = {'maxiter': 10000},  # maximum number of iterations
                   tol     = 0.0001,              # tolerance or final accuracy in the optimization 
                   args    = (weights, 
                              n_qbits, 
                              depth, 
                              shots,
                              cost,
                              algorithm,
                              alpha,
                              backend_name,
                              verbosity))    # the arguments of 'cost_function_cobyla', except 'params'

    # Time stops when the optimization stops https://qiskit.org/
    end_time = time.time()
    
    # Total time taken for the optimization
    elapsed_time = end_time - start_time 

    # Number of cost function evaluations during the optimization
    n_func_evaluations = res.nfev

    # Obtain the output distribution using the final parameters
    # VQE
    if algorithm == "VQE":
        optimal_circuit = VQE_circuit(res.x, 
                                      n_qbits, 
                                      depth)

    # Define the backend for the evaluation of the optimal circuit
    # - in case it is a simulator
    if backend_name == 'qasm_simulator':
        backend = Aer.get_backend('qasm_simulator')
    # - in case it is a real quantum device
    else:
        provider = QiskitRuntimeService()
        backend = provider.get_backend(backend_name)

    # Get the results from the circuit with the optimized parameters
    new_circuit = transpile(optimal_circuit, 
                            backend)

    job = backend.run(new_circuit,
                      shots = final_eval_shots)
    
    counts = job.result().get_counts(optimal_circuit)
    
    # The optimized rotation angles
    optimal_angles = res.x
    
    # The cost function of the optimal circuit
    final_cost = res.fun
    
    return elapsed_time, counts, shots, n_func_evaluations, final_eval_shots, optimal_angles, final_cost

In [14]:
2**21

2097152

In [25]:
final_eval_shots = 1024

weights = np.array([[0, 1, 0, 0, 0],
                    [1, 0, 3, 1, 0],
                    [0, 3, 0, 0, 2],
                    [0, 1, 0, 0, 2],
                    [0, 0, 2, 2, 0]])

time_vs_shots(weights, final_eval_shots, n_qbits=5,depth=4, verbosity = False)

(11.620121717453003,
 {'01001': 1,
  '11001': 15,
  '01010': 1,
  '11000': 3,
  '10100': 1,
  '10101': 12,
  '00110': 26,
  '01110': 1,
  '00101': 1,
  '10001': 6,
  '10110': 957},
 1024,
 286,
 1024,
 array([[ 1.16447173,  2.52241108,  1.59249025,  0.74789167,  1.37758357],
        [-0.04508625,  0.47127114, -0.14555245, -0.10019049,  1.5194394 ],
        [-0.42122118,  0.30665605,  0.8707306 ,  0.94513636,  0.78119993],
        [ 0.07422825, -0.36519337,  0.98307729, -0.78112931, -0.72489068],
        [-0.68457412, -0.04728897,  1.26211494, -0.64959998, -0.49491385]]),
 -8.88671875)

In [23]:
# Compute the value of the cost function of each eigenstate in a solution
# and returns to 'best candidate' eigenstate
# results_dict: the eigenstate-freq dictionary returned by 'time_vs_shots'
# weights: the original QUBO matrix
def best_candidate_finder(results_dict, 
                          weights):
        
    # the eigenstates obtained by the evaluation of the circuit
    eigenstates = list(results_dict.keys())
        
    # initialize the cost function
    min_cost = 0
    best_candidate = 0
    
    for k in range(len(eigenstates)):
        # ndarray of the digits extracted from the eigenstate string 
        x = np.array([int(num) for num in eigenstates[k]])
        # Cost function of to the k-th eigenstate
        cost = x.dot(weights.dot(1-x))
        print(cost)
        if cost >= min_cost:
            min_cost = cost
            best_candidate = eigenstates[k]
    
    return best_candidate

In [27]:
weights = np.array([[0, 1, 0, 0, 0],
                    [1, 0, 3, 1, 0],
                    [0, 3, 0, 0, 2],
                    [0, 1, 0, 0, 2],
                    [0, 0, 2, 2, 0]])

results_dict = time_vs_shots(weights, final_eval_shots, n_qbits=5,depth=4, verbosity = False)[1]

best_candidate_finder(results_dict, weights)

4
4
6
6
5
5
6
8
9


'01001'

In [28]:
results_dict 

{'01101': 1,
 '00001': 2,
 '10101': 1,
 '01011': 1,
 '01110': 1,
 '10001': 7,
 '01010': 5,
 '11001': 3,
 '01001': 1003}

Que significa el valor del coste?

In [18]:
# Function to compute F_opt
def F_opt_finder(results_obj,
                 weights,
                 opt_sol,
                 n_shots       = 1024,
                 n_eigenstates = 1000):
    """
    Returns the fraction of optimal solutions.
    
    Given the object returned by 'time_vs_shots',
    computes the fraction of best_candidates solutions
    which are optimal solutions.
    
    Inputs:
    results_obj: the object returned by 'time_vs_shots',
    n_shots: the 'number of shots' to investigate,
    W: the original QUBO matrix,
    opt_sol: list of the optimal solutions to the problem,
    n_eigenstates: maximum number of eigenstates in a solution.
    """
    # Initialize the counter of repetitions for the
    # selected number of shots
    N_rep = 0
    # Initialize the counter of best candidates which 
    # are optimal solutions    
    N_bc  = 0
    # Scan all the entries of the object
    for res in results_obj:
        print(res)
        # Select only the entries corresponding to 
        # the selected number of shots
        if res[2] == n_shots:
            # If the number of shots is the one we want to check,
            # sum 1 to the number of repetitions
            N_rep += 1
            # Find best candidate
            bc = best_candidate_finder(res[1], weights)
            # best candidate must contain the optimal solution
            print(bc)
            if bc in opt_sol:
                # best candidate must have less than 'n_eigenstates' eigenstates
                if len(res[1]) < n_eigenstates:
                    N_bc += 1
    # Initialize output value
    F_opt = 0
    # If N_rep is not 0, return the fraction of best candidates
    # which are also optimal solutions
    if N_rep != 0:
        F_opt = N_bc / N_rep
    else:
        print("The number of shots selected is not present")
    return F_opt

In [19]:
results_obj = [time_vs_shots(weights, 2**21, verbosity = False),
               time_vs_shots(weights, 2**10, verbosity = False)]
results_obj

[(1.6054978370666504,
  {'11': 2171, '00': 1, '10': 901876, '01': 1193104},
  1024,
  51,
  2097152,
  array([[1.70218169, 1.50501036],
         [0.74537481, 0.77557917]]),
  -0.9990234375),
 (1.5477209091186523,
  {'11': 2, '10': 132, '01': 890},
  1024,
  50,
  1024,
  array([[2.22158633, 1.02211471],
         [0.51544875, 0.91048187]]),
  -0.9951171875)]

Como saco opt_sol?

In [20]:
weights = np.array([[0, 1],
                    [1, 0]])

opt_sol = ['01']

F_opt_finder(results_obj, weights, opt_sol)

(1.6054978370666504, {'11': 2171, '00': 1, '10': 901876, '01': 1193104}, 1024, 51, 2097152, array([[1.70218169, 1.50501036],
       [0.74537481, 0.77557917]]), -0.9990234375)
10
(1.5477209091186523, {'11': 2, '10': 132, '01': 890}, 1024, 50, 1024, array([[2.22158633, 1.02211471],
       [0.51544875, 0.91048187]]), -0.9951171875)
10


0.0